# EMPATHY TRACK AS CLASSIFICATION - GPT-3 Fine-Tuning
## ACL 2023 Conference
## WASSA 2023 Shared Task on Empathy, Emotion, and Personality Detection in Interactions
More details [here](https://codalab.lisn.upsaclay.fr/competitions/11167#learn_the_details)

In [92]:
import numpy as np
import pandas as pd
import re, os
import ftfy
import pycld2 as cld2
import time, json
import backoff
import tiktoken

import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)
#os.path.join()

In [43]:
encoding = tiktoken.encoding_for_model("davinci")
len(encoding.encode('102'))

1

In [2]:
multi_spaces = re.compile('\s{2,}')

def clean_text(s):
    if not isinstance(s, str):
        return s
    for char in ['�', '•']:
        if char in s:
            s = s.replace(char, ' ')
    s = ftfy.fix_text(s)
    
    #s = clean.sub(' ', s.lower())
    s = multi_spaces.sub(' ', s)
        
    return s.strip()

In [3]:
def detect_lang( t ):
    '''
        Return the language(s) in string s.
        Naive Bayes classifier under the hood -
        results are less certain for strings that are too short.
        Returns up to three languages with confidence scores.
        More on usage: https://pypi.org/project/pycld2/
    '''
    _, _, details = cld2.detect( ftfy.fix_text( t ) )
    return details[0][0]

# Load and prepare data
### a) Training set

In [4]:
# project files
wdir  = 'data'
files = [ 'WASSA23_essay_level_with_labels_train.tsv', 'WASSA23_essay_level_dev.tsv',
          'WASSA23_conv_level_with_labels_train.tsv', 'WASSA23_conv_level_dev.tsv',
          'articles_adobe_AMT.csv', 'goldstandard_CONV_dev.tsv', 'goldstandard_dev.tsv', ]

In [5]:
file = 'data/df_train.pkl'
df_train = pd.read_pickle(file)

2023-05-10 00:32:05.743574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
file = 'data/df_dev.pkl'
df_dev = pd.read_pickle(file)

In [6]:
df_full = pd.read_csv( os.path.join(wdir, files[0]), sep='\t' )
df_full['essay_clean'] = df_full['essay'].apply(clean_text)


print(df_full.shape, '\n')
print(df_full.dtypes, '\n')
print(df_full.isna().sum(), 'n')

print('\nTotal essays: ', df_full.shape[0])
print('Unique essays:', len(df_full['essay_clean'].unique()))
print('Unique essay IDs:', len(df_full['essay_id'].unique()))
df_full.head(25)

(792, 25) 

conversation_id                    int64
article_id                         int64
essay                             object
empathy                          float64
distress                         float64
speaker_id                         int64
gender                            object
education                         object
race                              object
age                               object
income                            object
personality_conscientiousness     object
personality_openess               object
personality_extraversion          object
personality_agreeableness         object
personality_stability             object
iri_perspective_taking            object
iri_personal_distress             object
iri_fantasy                       object
iri_empathatic_concern            object
speaker_number                     int64
split                             object
essay_id                           int64
emotion                           object
essa

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,income,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,speaker_number,split,essay_id,emotion,essay_clean
0,2,35,It breaks my heart to see people living in tho...,6.833333,6.625,30,1,6,3,37,40000,7.0,5.5,1.0,6.5,6.0,4.857,2.0,3.429,5.0,1,train,1,Hope/Sadness,It breaks my heart to see people living in tho...
1,3,35,I wonder why there aren't more people trying t...,5.833333,6.000,19,1,6,2,32,35000,5.5,5.0,2.0,5.5,4.5,3.429,2.857,2.857,2.714,1,train,2,Anger,I wonder why there aren't more people trying t...
2,5,35,"After reading the article, you can't help but ...",1.000000,1.375,17,1,6,1,29,85000,6.75,6.75,6.75,6.75,7.0,4.643,2.0715,4.143,4.643,1,train,4,Sadness,"After reading the article, you can't help but ..."
3,6,213,It is so sad that someone who had such an amaz...,6.166667,6.625,16,2,5,1,28,50000,6.0,6.0,5.0,4.5,3.5,5.0,4.143,4.857,5.0,1,train,5,Sadness,It is so sad that someone who had such an amaz...
4,8,213,"From reading the article, it looks like the wo...",6.833333,1.000,30,1,6,3,37,40000,7.0,5.5,1.0,6.5,6.0,4.857,2.0,3.429,5.0,1,train,7,Neutral,"From reading the article, it looks like the wo..."
5,10,213,That's sad. Regardless of what they find out ...,1.666667,1.125,49,1,5,1,31,82000,3.5,2.5,2.0,4.5,5.0,3.571,2.286,1.857,2.0,1,train,9,Sadness,That's sad. Regardless of what they find out h...
6,11,78,"After reading the article, my reaction is that...",1.500000,1.000,17,1,6,1,29,85000,6.75,6.75,6.75,6.75,7.0,4.643,2.0715,4.143,4.643,1,train,10,Sadness,"After reading the article, my reaction is that..."
7,13,78,It sounds like these boys had a really rough l...,2.000000,1.000,24,2,7,1,38,42000,7.0,3.5,6.5,5.5,6.5,3.429,2.714,2.571,3.857,1,train,12,Sadness,It sounds like these boys had a really rough l...
8,14,78,This is a tragic and sad story about how some ...,6.000000,3.000,43,2,6,1,33,110000,7.0,4.5,1.5,7.0,7.0,4.286,1.286,3.857,4.0,1,train,13,Sadness,This is a tragic and sad story about how some ...
9,17,336,Hello. I feel really terrible about the curren...,7.000000,1.000,31,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,1,train,16,Disgust/Sadness,Hello. I feel really terrible about the curren...


__Based on the above stats, there are no duplicate essays, no NaN values, but there is missing data in the form of a string instead of a numerical value - the presence of `unknown` in some numerical columns (gender, education, etc.) changes their type from `integer` / `float` to `object`__

In [14]:
# add missing columns
col_names = [c for c in df_full.columns if c not in df_train.columns]
for col in col_names:
    df_train[col] = df_full[col].values

In [19]:
df_dev.columns

Index(['article_id', 'conversation_id', 'speaker_number', 'essay_id',
       'speaker_id', 'essay', 'essay_clean', 'split', 'gender', 'education',
       'race', 'age', 'income', 'emotion', 'emotion_count', 'char_length',
       'word_length', 'target_encoded', 'article', 'article_clean',
       'essay_clean_docs', 'essay_clean_spellchecked', 'article_clean_docs',
       'article_clean_spellchecked', 'compare1', 'compare2', 'gpt_embedding',
       'closest_texts', 'emotion_no_2nd_neut', 'gpt35_keywords', 'gpt35_title',
       'gpt35_summary', 'gpt4_title', 'gpt4_summary', 'gpt4_keywords',
       'davinci_preds2', 'davinci_preds_converted2', 'davinci_preds_encoded2',
       'davinci_preds3', 'davinci_preds_converted3', 'davinci_preds_encoded3',
       'empathy', 'distress', 'personality_conscientiousness',
       'personality_openess', 'personality_extraversion',
       'personality_agreeableness', 'personality_stability',
       'iri_perspective_taking', 'iri_personal_distress', 'iri_f

In [18]:
cols_to_drop = [ 'target_encoded2', 'compare', 'char_length', 'word_length', 'essay_clean_docs',
                 'article_clean_docs', 'compare1', 'compare2', ]
df_train = df_train.drop( cols_to_drop, axis=1)

In [20]:
cols_to_drop = [ 'char_length', 'word_length', 'essay_clean_docs',
                 'article_clean_docs', 'compare1', 'compare2', ]
df_dev = df_dev.drop( cols_to_drop, axis=1 )

In [21]:
file = 'data/df_train.pkl'
df_train.to_pickle( file )

file = 'data/df_dev.pkl'
df_dev.to_pickle( file )

### b) Dev + Gold

In [9]:
colnames=[ 'empathy', 'distress', 'personality_conscientiousness', 'personality_openess',
           'personality_extraversion', 'personality_agreeableness',
           'personality_stability', 'iri_perspective_taking',
           'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern', ] 
print(len(colnames))
df_gold = pd.read_csv( os.path.join(wdir, files[6]), names=colnames, sep='\t')
print(df_essay_test.shape)
print(df_gold.shape, '\n')
print(df_gold.isna().sum(), '\n')
print(df_gold.dtypes, '\n')
df_gold.head(25)

12
(208, 13)
(208, 12) 

empathy                          0
distress                         0
emotion                          0
personality_conscientiousness    0
personality_openess              0
personality_extraversion         0
personality_agreeableness        0
personality_stability            0
iri_perspective_taking           0
iri_personal_distress            0
iri_fantasy                      0
iri_empathatic_concern           0
dtype: int64 

empathy                          float64
distress                         float64
emotion                           object
personality_conscientiousness    float64
personality_openess              float64
personality_extraversion         float64
personality_agreeableness        float64
personality_stability            float64
iri_perspective_taking           float64
iri_personal_distress            float64
iri_fantasy                      float64
iri_empathatic_concern           float64
dtype: object 



,empathy,distress,emotion,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,3.833333,3.375,Sadness,5.00,3.00,5.00,4.00,3.5,2.714,3.0000,3.143,3.286
1,3.000000,1.000,Sadness,6.50,7.00,3.50,4.50,7.0,3.714,1.0000,2.429,1.429
2,3.833333,4.250,Sadness,5.00,3.00,5.00,4.00,3.5,2.714,3.0000,3.143,3.286
3,3.166667,2.375,Neutral,5.50,5.50,3.50,4.50,4.0,3.571,2.8570,3.571,3.143
4,3.333333,3.500,Sadness,5.00,3.00,5.00,4.00,3.5,2.714,3.0000,3.143,3.286
5,1.500000,1.500,Sadness,6.75,6.75,6.75,6.75,7.0,4.643,2.0715,4.143,4.643
6,6.000000,6.000,Neutral,3.00,4.00,6.00,6.00,6.5,3.429,2.8570,4.571,4.000
7,2.500000,1.000,Sadness,7.00,3.50,6.50,5.50,6.5,3.429,2.7140,2.571,3.857
8,4.000000,5.500,Sadness,6.00,6.00,5.50,6.50,3.0,4.857,3.1430,2.571,4.857
9,3.666667,2.250,Neutral,6.00,6.00,5.50,6.50,3.0,4.857,3.1430,2.571,4.857


In [12]:
# merge dev set and labels
for col in colnames:
    df_dev[col] = df_gold[col].values

df_dev.head()

,article_id,conversation_id,speaker_number,essay_id,speaker_id,essay,essay_clean,split,gender,education,race,age,income,emotion,emotion_count,char_length,word_length,target_encoded,article,article_clean,essay_clean_docs,essay_clean_spellchecked,article_clean_docs,article_clean_spellchecked,compare1,compare2,gpt_embedding,closest_texts,emotion_no_2nd_neut,gpt35_keywords,gpt35_title,gpt35_summary,gpt4_title,gpt4_summary,gpt4_keywords,davinci_preds2,davinci_preds_converted2,davinci_preds_encoded2,davinci_preds3,davinci_preds_converted3,davinci_preds_encoded3,empathy,distress,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,35,1,1,0,68,How sad is it that this kind of pain and suffe...,How sad is it that this kind of pain and suffe...,dev,2,2,1,21,20000,Sadness,1,339,63,"[0, 0, 0, 0, 0, 0, 1, 0]","A month after Hurricane Matthew, 800,000 Haiti...","A month after Hurricane Matthew, 800,000 Haiti...","(How, sad, is, it, that, this, kind, of, pain,...",How sad is it that this kind of pain and suffe...,"(A, month, after, Hurricane, Matthew, ,, 800,0...","A month after Hurricane Matthew, 800,000 Haiti...",False,False,"[0.01686943881213665, -0.00237144879065454, 0....",[This situation in Haiti is terrible and sadde...,[Sadness],"pain, suffering, living conditions, developed ...",The Disparity Between Developed Countries and ...,The author expresses sadness over the existenc...,The Stark Contrast Between Wealth and Poverty ...,The text highlights the disparity between deve...,"sad, pain, suffering, living conditions, exist...","[[0.8797003859451549, Sadness], [0.07585283480...","[Sadness, Disgust]","[0, 1, 0, 0, 0, 0, 1, 0]","[[0.9630779398840441, Sadness], [0.02382719985...","[Sadness, Disgust]","[0, 1, 0, 0, 0, 0, 1, 0]",3.833333,3.375,5.0,3.0,5.0,4.0,3.5,2.714,3.000,3.143,3.286
1,35,4,1,3,79,The article is kind of tragic and hits close t...,The article is kind of tragic and hits close t...,dev,1,6,3,33,64000,Sadness,1,367,63,"[0, 0, 0, 0, 0, 0, 1, 0]","A month after Hurricane Matthew, 800,000 Haiti...","A month after Hurricane Matthew, 800,000 Haiti...","(The, article, is, kind, of, tragic, and, hits...",The article is kind of tragic and hits close t...,"(A, month, after, Hurricane, Matthew, ,, 800,0...","A month after Hurricane Matthew, 800,000 Haiti...",True,False,"[-0.007789629977196455, 0.011638623662292957, ...","[In the 21st century, people are still starvin...",[Sadness],"Haitian immigrants, Haiti, problems, natural d...","""Developing Infrastructure: The Key to Helping...","The author, who is the son of Haitian immigran...",Haitian Infrastructure Development: A Long-Ter...,The text discusses the tragic situation in Hai...,"article, tragic, close to home, son, Haitian i...","[[0.8202061810703213, Neutral], [0.16278792482...","[Neutral, Sadness]","[0, 0, 0, 0, 0, 1, 1, 0]","[[0.5558798251000321, Sadness], [0.42004855384...","[Sadness, Neutral]","[0, 0, 0, 0, 0, 1, 1, 0]",3.000000,1.000,6.5,7.0,3.5,4.5,7.0,3.714,1.000,2.429,1.429
2,213,7,1,6,68,"I think that these kinds of stories, are sad, ...","I think that these kinds of stories, are sad, ...",dev,2,2,1,21,20000,Sadness,1,315,57,"[0, 0, 0, 0, 0, 0, 1, 0]",Miami Marlins star pitcher Jose Fernandez kill...,Miami Marlins star pitcher Jose Fernandez kill...,"(I, think, that, these, kinds, of, stories, ,,...","I think that these kinds of stories, are sad, ...","(Miami, Marlins, star, pitcher, Jose, Fernande...",Miami Marlins star pitcher Jose Fernandez kill...,True,False,"[-0.004882862325757742, -0.016483262181282043,...",[It is so sad that someone who had such an ama...,[Sadness],"stories, sad, inspirational, good feeling, coo...",The Inspiring Story of Overcoming Circumstances,The text discusses how stories that are sad bu...,Overcoming Adversity: The Power of Inspiration...,The text describes a story as sad but also ins...,"sa

# Convert numerical values into classification labels

In [33]:
print(df_train['empathy'].value_counts())
train_empathy = sorted([i for i in df_train['empathy'].unique()])
print(len(train_empathy))
print(train_empathy)

1.000000    97
6.000000    94
5.833333    44
7.000000    38
6.666667    37
4.000000    35
3.833333    28
6.166667    26
5.500000    25
4.166667    24
5.000000    22
5.666667    22
1.500000    20
6.500000    20
2.000000    17
4.833333    16
3.666667    15
1.166667    14
4.333333    14
1.333333    14
6.333333    13
2.500000    13
3.166667    12
1.666667    12
5.166667    11
6.833333    11
1.833333    11
3.000000    10
3.333333    10
5.333333    10
2.166667    10
3.500000    10
4.666667    10
2.333333     9
2.666667     7
4.500000     6
2.833333     5
Name: empathy, dtype: int64
37
[1.0, 1.1666666666666667, 1.3333333333333333, 1.5, 1.6666666666666667, 1.8333333333333333, 2.0, 2.1666666666666665, 2.333333333333333, 2.5, 2.6666666666666665, 2.833333333333333, 3.0, 3.1666666666666665, 3.333333333333333, 3.5, 3.6666666666666665, 3.8333333333333335, 4.0, 4.166666666666667, 4.333333333333333, 4.5, 4.666666666666667, 4.833333333333333, 5.0, 5.166666666666667, 5.333333333333333, 5.5, 5.6666666666

In [44]:
print(df_dev['empathy'].value_counts())
dev_empathy = sorted([i for i in df_dev['empathy'].unique()])
print(len(dev_empathy))
print(dev_empathy)

6.000000    24
4.000000    14
3.833333    13
1.000000    13
3.500000    12
7.000000    11
3.666667    11
4.166667     9
6.666667     6
4.666667     6
4.333333     6
5.833333     6
5.000000     6
6.500000     6
3.166667     6
6.333333     6
3.000000     5
3.333333     5
5.166667     4
4.500000     4
5.500000     4
4.833333     4
1.666667     4
2.000000     3
5.666667     3
1.333333     2
2.166667     2
2.333333     2
1.166667     2
2.666667     2
2.500000     2
5.333333     2
1.833333     1
6.166667     1
1.500000     1
Name: empathy, dtype: int64
35
[1.0, 1.1666666666666667, 1.3333333333333333, 1.5, 1.6666666666666667, 1.8333333333333333, 2.0, 2.1666666666666665, 2.333333333333333, 2.5, 2.6666666666666665, 3.0, 3.1666666666666665, 3.333333333333333, 3.5, 3.6666666666666665, 3.8333333333333335, 4.0, 4.166666666666667, 4.333333333333333, 4.5, 4.666666666666667, 4.833333333333333, 5.0, 5.166666666666667, 5.333333333333333, 5.5, 5.666666666666667, 5.833333333333333, 6.0, 6.166666666666667,

In [45]:
print(df_train['distress'].value_counts())
train_distress = sorted([i for i in df_train['distress'].unique()])
print(len(train_distress))
print(train_distress)

1.000    83
6.000    77
2.000    30
1.750    26
4.000    24
6.250    24
1.625    24
1.500    23
1.250    22
1.125    21
6.125    20
5.875    20
1.375    19
7.000    19
6.625    18
6.750    17
3.875    16
5.750    15
6.375    14
6.500    14
2.625    14
5.500    13
2.125    13
2.250    12
4.750    12
3.000    12
4.625    11
2.375    11
3.625    11
5.000    11
4.375    11
2.750    11
5.625    10
4.125    10
4.250     9
3.125     9
5.125     9
2.875     8
5.375     8
1.875     8
3.500     7
3.375     7
4.875     6
3.250     6
5.250     6
2.500     6
3.750     6
4.500     5
6.875     4
Name: distress, dtype: int64
49
[1.0, 1.125, 1.25, 1.375, 1.5, 1.625, 1.75, 1.875, 2.0, 2.125, 2.25, 2.375, 2.5, 2.625, 2.75, 2.875, 3.0, 3.125, 3.25, 3.375, 3.5, 3.625, 3.75, 3.875, 4.0, 4.125, 4.25, 4.375, 4.5, 4.625, 4.75, 4.875, 5.0, 5.125, 5.25, 5.375, 5.5, 5.625, 5.75, 5.875, 6.0, 6.125, 6.25, 6.375, 6.5, 6.625, 6.75, 6.875, 7.0]


In [46]:
print(df_dev['distress'].value_counts())
dev_distress = sorted([i for i in df_dev['distress'].unique()])
print(len(dev_distress))
print(dev_distress)

6.000    20
1.000    16
3.625    11
3.500    10
4.250     9
4.000     7
3.375     6
3.250     6
3.750     6
1.625     6
7.000     6
2.000     6
3.875     5
2.250     5
5.750     5
1.250     5
5.625     5
1.375     5
6.250     5
2.875     4
2.625     4
3.000     4
2.375     4
6.125     3
2.125     3
5.000     3
1.875     3
5.250     3
4.500     3
5.125     3
6.500     3
3.125     3
4.750     3
2.750     2
1.500     2
4.125     2
6.625     2
6.375     2
6.750     1
5.875     1
2.500     1
5.500     1
4.625     1
4.875     1
1.750     1
4.375     1
Name: distress, dtype: int64
46
[1.0, 1.25, 1.375, 1.5, 1.625, 1.75, 1.875, 2.0, 2.125, 2.25, 2.375, 2.5, 2.625, 2.75, 2.875, 3.0, 3.125, 3.25, 3.375, 3.5, 3.625, 3.75, 3.875, 4.0, 4.125, 4.25, 4.375, 4.5, 4.625, 4.75, 4.875, 5.0, 5.125, 5.25, 5.5, 5.625, 5.75, 5.875, 6.0, 6.125, 6.25, 6.375, 6.5, 6.625, 6.75, 7.0]


In [54]:
emp2label = dict(zip(train_empathy, list(map(str, range(100, 1000)))))
label2emp = {v: k for k,v in emp2label.items()}
print('emp2label =', empathy_map, '\n')
print('label2emp =', label2emp)

emp2label = {1.0: '100', 1.1666666666666667: '101', 1.3333333333333333: '102', 1.5: '103', 1.6666666666666667: '104', 1.8333333333333333: '105', 2.0: '106', 2.1666666666666665: '107', 2.333333333333333: '108', 2.5: '109', 2.6666666666666665: '110', 2.833333333333333: '111', 3.0: '112', 3.1666666666666665: '113', 3.333333333333333: '114', 3.5: '115', 3.6666666666666665: '116', 3.8333333333333335: '117', 4.0: '118', 4.166666666666667: '119', 4.333333333333333: '120', 4.5: '121', 4.666666666666667: '122', 4.833333333333333: '123', 5.0: '124', 5.166666666666667: '125', 5.333333333333333: '126', 5.5: '127', 5.666666666666667: '128', 5.833333333333333: '129', 6.0: '130', 6.166666666666667: '131', 6.333333333333333: '132', 6.5: '133', 6.666666666666667: '134', 6.833333333333333: '135', 7.0: '136'} 

label2emp = {'100': 1.0, '101': 1.1666666666666667, '102': 1.3333333333333333, '103': 1.5, '104': 1.6666666666666667, '105': 1.8333333333333333, '106': 2.0, '107': 2.1666666666666665, '108': 2.333

In [55]:
dist2label = dict(zip(train_distress, list(map(str, range(100, 1000)))))
label2dist = {v: k for k,v in dist2label.items()}
print('dist2label =', dist2label, '\n')
print('label2dist =', label2dist)

dist2label = {1.0: '100', 1.125: '101', 1.25: '102', 1.375: '103', 1.5: '104', 1.625: '105', 1.75: '106', 1.875: '107', 2.0: '108', 2.125: '109', 2.25: '110', 2.375: '111', 2.5: '112', 2.625: '113', 2.75: '114', 2.875: '115', 3.0: '116', 3.125: '117', 3.25: '118', 3.375: '119', 3.5: '120', 3.625: '121', 3.75: '122', 3.875: '123', 4.0: '124', 4.125: '125', 4.25: '126', 4.375: '127', 4.5: '128', 4.625: '129', 4.75: '130', 4.875: '131', 5.0: '132', 5.125: '133', 5.25: '134', 5.375: '135', 5.5: '136', 5.625: '137', 5.75: '138', 5.875: '139', 6.0: '140', 6.125: '141', 6.25: '142', 6.375: '143', 6.5: '144', 6.625: '145', 6.75: '146', 6.875: '147', 7.0: '148'} 

label2dist = {'100': 1.0, '101': 1.125, '102': 1.25, '103': 1.375, '104': 1.5, '105': 1.625, '106': 1.75, '107': 1.875, '108': 2.0, '109': 2.125, '110': 2.25, '111': 2.375, '112': 2.5, '113': 2.625, '114': 2.75, '115': 2.875, '116': 3.0, '117': 3.125, '118': 3.25, '119': 3.375, '120': 3.5, '121': 3.625, '122': 3.75, '123': 3.875, '124

In [69]:
df_train['empathy_label'] = df_train['empathy'].map( emp2label )
df_dev['empathy_label']   = df_dev['empathy'].map( emp2label )

df_train['distress_label'] = df_train['distress'].map( dist2label )
df_dev['distress_label']   = df_dev['distress'].map( dist2label )

# Save data for fine-tuning

In [83]:
dict_list      = []
text_col       = 'essay_clean'
target_col     = 'distress_label'
prompt_end     = '\n\n###\n\n'
completion_end = '\n###'
for prompt, completion in df_train[[text_col, target_col]].values:
    dict_list.append({ "prompt":     prompt + prompt_end,
                       "completion": ' ' + completion + completion_end })

In [86]:
file = 'data/data_distress.jsonl'
with open(file, 'w') as f:
    for entry in dict_list:
        json.dump(entry, f)
        f.write('\n')

# Fine-tune DaVinci models

`openai api fine_tunes.create -t data_empathy.jsonl -m davinci --suffix "train_empathy"`  
`openai api fine_tunes.create -t data_distress.jsonl -m davinci --suffix "train_distress"`

In [88]:
# cancel fine-tune; creating fine-tunes can be don this way too
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
ft_id = "ft-kly7Pn2M6mEQsrJPuncc26mD"
openai.FineTune.cancel(id=ft_id)

<FineTune fine-tune id=ft-kly7Pn2M6mEQsrJPuncc26mD at 0x7f9d7fc7f560> JSON: {
  "created_at": 1683703729,
  "events": [
    {
      "created_at": 1683703729,
      "level": "info",
      "message": "Created fine-tune: ft-kly7Pn2M6mEQsrJPuncc26mD",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1683704880,
      "level": "info",
      "message": "Fine-tune cancelled",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-kly7Pn2M6mEQsrJPuncc26mD",
  "model": "davinci",
  "object": "fine-tune",
  "organization_id": "org-YgKrsHc2rA34WUg4e5lPF38A",
  "result_files": [],
  "status": "cancelled",
  "training_files": [
    {
      "bytes": 372866,
      "created_at": 1683703728,
      "filename": "data_distress.jsonl",
      "id": "file-uFUOKjcCNnFuKrPoZyasxMW2",
      "object": "file",
      "purpose": "fine

## Dev set results for empathy

In [89]:
openai.api_key = os.getenv("OPENAI_API_KEY")
model = "davinci:ft-tovoz-rosnim:train-empathy-2023-05-10-07-41-15"

In [93]:
@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_time=10)
def get_fine_tuned_response(model_, prompt_, temperature=0, max_tokens=None):
    '''Send request, return reponse'''
    response  = openai.Completion.create(
        model = model_,
        prompt = prompt_,       
        temperature=0,
        logprobs=5,
        max_tokens=1,        
        #suffix=completion_end,        
        #top_p=1,
        #n=1,    # how many completions to gerenerate
        #presence_penalty=0,
        #frequency_penalty=0,
        #best_of=1,
        #stream = False,
        #stop=None, 
        #logit_bias=None,
    )
    #content = response['choices'][0]['message']['content'].strip()    
    return response

In [95]:
# one example
idx = 5
example      = df_dev[text_col].values[idx]
response     = get_fine_tuned_response(model, example + prompt_end)
top_logprobs = response["choices"][0]["logprobs"]["top_logprobs"][0]
print('RESPONSE:\n', response, '\n')

probs = sorted([[np.exp(logp), token.strip()] for token, logp in top_logprobs.items()], reverse=True)
#probs = [[a, pred2label.get(b)] for a,b in probs]
print(probs)

text = response["choices"][0]["text"]
print(text.strip())

RESPONSE:
 {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          387
        ],
        "token_logprobs": [
          -0.044179264
        ],
        "tokens": [
          " 100"
        ],
        "top_logprobs": [
          {
            " 100": -0.044179264,
            " 101": -8.148627,
            " 102": -7.677371,
            " 103": -3.346757,
            " 104": -4.959752
          }
        ]
      },
      "text": " 100"
    }
  ],
  "created": 1683705022,
  "id": "cmpl-7EYsYISHTtC4G3omSnj5kXA2C9Pqm",
  "model": "davinci:ft-tovoz-rosnim:train-empathy-2023-05-10-07-41-15",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 78,
    "total_tokens": 79
  }
} 

[[0.9567824254540315, '100'], [0.03519831735272724, '103'], [0.007014667254367292, '104'], [0.0004631910286928621, '102'], [0.0002891320655532702, '101']]
100


In [96]:
# apply to all examples from dev set
raw_response   = dict()
probs_response = dict()
text_response  = dict()

count = 0
for example in df_dev[text_col].values:
    if example in probs_response:
        continue
        
    response = get_fine_tuned_response(model, example + prompt_end)
    
    top_logprobs = response["choices"][0]["logprobs"]["top_logprobs"][0]
    probs = sorted([[np.exp(logp), token.strip()] for token, logp in top_logprobs.items()], reverse=True)
    #print(probs)

    text = response["choices"][0]["text"].strip()
    #print(text.strip())
    
    raw_response[ example ]   = response
    probs_response[ example ] = probs
    text_response[ example ]  = text
    
    count += 1
    if count%10==0:
        print(f'Processed {count} essays')

Processed 10 essays
Processed 20 essays
Processed 30 essays
Processed 40 essays
Processed 50 essays
Processed 60 essays
Processed 70 essays
Processed 80 essays
Processed 90 essays
Processed 100 essays
Processed 110 essays
Processed 120 essays
Processed 130 essays
Processed 140 essays
Processed 150 essays
Processed 160 essays
Processed 170 essays
Processed 180 essays
Processed 190 essays
Processed 200 essays


In [98]:
len(raw_response), len(probs_response), len(text_response)

(208, 208, 208)

In [99]:
df_dev['empathy_preds'] = df_dev[text_col].map( probs_response )

In [100]:
# compare groundtruth and predictions
for a, b in df_dev[[ text_col, 'empathy_preds']].values:
    print(f"Text: {a}")
    print(f"Predicted: {b}")
    print('\n', '='*77, '\n')

Text: How sad is it that this kind of pain and suffering, and those kind of living conditions still exsist today? what a gap we have in society between developed countries and those that aren't. It's crazy to drive around the US and see all the money people spend on pointless things, and then to think about how the people in Haiti are living.
Predicted: [[0.33251234225873494, '119'], [0.25116826147550164, '117'], [0.1117502844586973, '112'], [0.0646209898945953, '113'], [0.05713538796987621, '115']]


Text: The article is kind of tragic and hits close to home as I am the son of Haitian immigrants. Haiti has a lot of problems that only become exaggerated during natural disasters. I think what the Haitian people really need from the international community is help developing infrastructure so they can address these issues themselves. Foreign aid only acts as a band aid.
Predicted: [[0.2984017339037529, '128'], [0.18822368680505006, '129'], [0.1557357684199864, '127'], [0.0938253691046217

In [102]:
# convert back to numeraical value
df_dev['empathy_preds_converted'] = df_dev['empathy_preds'].apply( lambda x: x[0][1] )
df_dev['empathy_preds_numeric']   = df_dev['empathy_preds_converted'].map( label2emp )

In [111]:
print(df_dev['empathy'].corr(df_dev['empathy_preds_numeric'], method='pearson'))

0.5449922695703181


In [109]:
file = 'data/df_train.pkl'
df_train.to_pickle( file )

file = 'data/df_dev.pkl'
df_dev.to_pickle( file )

## Dev set results for distress

In [112]:
openai.api_key = os.getenv("OPENAI_API_KEY")
model = "davinci:ft-tovoz-rosnim:train-distress-2023-05-10-08-31-29"

In [113]:
@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_time=10)
def get_fine_tuned_response(model_, prompt_, temperature=0, max_tokens=None):
    '''Send request, return reponse'''
    response  = openai.Completion.create(
        model = model_,
        prompt = prompt_,       
        temperature=0,
        logprobs=5,
        max_tokens=1,        
        #suffix=completion_end,        
        #top_p=1,
        #n=1,    # how many completions to gerenerate
        #presence_penalty=0,
        #frequency_penalty=0,
        #best_of=1,
        #stream = False,
        #stop=None, 
        #logit_bias=None,
    )
    #content = response['choices'][0]['message']['content'].strip()    
    return response

In [114]:
# one example
idx = 5
example      = df_dev[text_col].values[idx]
response     = get_fine_tuned_response(model, example + prompt_end)
top_logprobs = response["choices"][0]["logprobs"]["top_logprobs"][0]
print('RESPONSE:\n', response, '\n')

probs = sorted([[np.exp(logp), token.strip()] for token, logp in top_logprobs.items()], reverse=True)
#probs = [[a, pred2label.get(b)] for a,b in probs]
print(probs)

text = response["choices"][0]["text"]
print(text.strip())

RESPONSE:
 {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          387
        ],
        "token_logprobs": [
          -1.129655
        ],
        "tokens": [
          " 104"
        ],
        "top_logprobs": [
          {
            " 100": -2.496457,
            " 103": -2.698331,
            " 104": -1.129655,
            " 105": -1.1603634,
            " 106": -2.3981104
          }
        ]
      },
      "text": " 104"
    }
  ],
  "created": 1683708465,
  "id": "cmpl-7EZm5axPiPfD2dgUAx4KR82suJ3KA",
  "model": "davinci:ft-tovoz-rosnim:train-distress-2023-05-10-08-31-29",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 78,
    "total_tokens": 79
  }
} 

[[0.32314472212244627, '104'], [0.31337228070138895, '105'], [0.09088953599423341, '106'], [0.08237634158331059, '100'], [0.06731777239513598, '103']]
104


In [115]:
# apply to all examples from dev set
raw_response   = dict()
probs_response = dict()
text_response  = dict()

count = 0
for example in df_dev[text_col].values:
    if example in probs_response:
        continue
        
    response = get_fine_tuned_response(model, example + prompt_end)
    
    top_logprobs = response["choices"][0]["logprobs"]["top_logprobs"][0]
    probs = sorted([[np.exp(logp), token.strip()] for token, logp in top_logprobs.items()], reverse=True)
    #print(probs)

    text = response["choices"][0]["text"].strip()
    #print(text.strip())
    
    raw_response[ example ]   = response
    probs_response[ example ] = probs
    text_response[ example ]  = text
    
    count += 1
    if count%10==0:
        print(f'Processed {count} essays')

Processed 10 essays
Processed 20 essays
Processed 30 essays
Processed 40 essays
Processed 50 essays
Processed 60 essays
Processed 70 essays
Processed 80 essays
Processed 90 essays
Processed 100 essays
Processed 110 essays
Processed 120 essays
Processed 130 essays
Processed 140 essays
Processed 150 essays
Processed 160 essays
Processed 170 essays
Processed 180 essays
Processed 190 essays
Processed 200 essays


In [116]:
len(raw_response), len(probs_response), len(text_response)

(208, 208, 208)

In [117]:
df_dev['distress_preds'] = df_dev[text_col].map( probs_response )

In [118]:
# convert back to numeraical value
df_dev['distress_preds_converted'] = df_dev['distress_preds'].apply( lambda x: x[0][1] )
df_dev['distress_preds_numeric']   = df_dev['distress_preds_converted'].map( label2emp )

In [119]:
print(df_dev['distress'].corr(df_dev['distress_preds_numeric'], method='pearson'))

0.34320756382666395


In [122]:
file = 'data/df_train.pkl'
df_train.to_pickle( file )

file = 'data/df_dev.pkl'
df_dev.to_pickle( file )